In [ ]:
import requests
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from shapely.geometry import Point
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, LayersControl, GeoData, basemaps, Heatmap, LegendControl

In [ ]:
## I chose to create an API object, to include the different sensor systems as separate layers in the map:
class Firms_source_object:
    ''' FIRMS API Objects
    FIRMS (Fire Information for Resource Management System) is NASAs open data platform for wildfire observation.
    They provide several sensors with various days.
    Source: https://firms.modaps.eosdis.nasa.gov/
    '''
    def __init__(self, api_token: str, sensor_source: str, day_range = 1, base_url = "https://firms.modaps.eosdis.nasa.gov/api/area/csv"):
        self.base_url = base_url
        self.__api_token = api_token
        self.sensor_source = sensor_source
        self.day_range = day_range
        self.__api = f'{self.base_url}/{self.__api_token}/{self.sensor_source}/world/{self.day_range}' ## creates the original url: "https://firms.modaps.eosdis.nasa.gov/api/area/csv/36ece617d2f514d4e90f01418d1b9e28/VIIRS_SNPP_NRT/world/1"


    def get_data(self):
        '''The API returns a csv file, which can directely be opened with panda'''
        self.data = pd.read_csv(self.__api) ## Makes API call
        self.api_timestamp = datetime.now() ## Time when API call was made
        return self.data


    def is_uptodate(self):
        if hasattr(self, "api_timestamp"):
            time_delta = datetime.now() - self.api_timestamp
            if time_delta < timedelta(minutes = 15):
                return True
        return False




class Wildfire_layer:
    def __init__(self,firms_source: Firms_source_object, layer_name: str, fill_color = "red"):
        self.firms_source = firms_source
        self.data = self.retrieve_data(self.firms_source)
        self.layer = GeoData(geo_dataframe = self.data,
                            name = layer_name,
                            style={'color': "black",'fillColor':fill_color, 'opacity':1, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6, 'radius':8},
                            hover_style={'fillColor': 'yellow' , 'fillOpacity': 0.2},
                            point_style = {'radius': 3, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': fill_color, 'weight': 3},
                            )


    def retrieve_data(self, firms_object): ##Firms Object, Call happen with retrieve_data(self.firms_source)
        if firms_object.is_uptodate():
            if hasattr(self, "data_clean"):
                return self.data_clean
            else: # API Data is Up-to-date, but data_clean is missing -> don't make new API Call, just clean internal data
                self.data_clean = self.clean_data(firms_object.data) 
                return self.data_clean
        # If API Data is not Up-to-date: api.get_data()        
        data_raw = firms_object.get_data()
        self.data_clean = self.clean_data(data_raw)
        return self.data_clean


    def clean_data(self, data_raw):
        data_clean = data_raw
        data_clean["geometry"] = [Point(xy) for xy in zip(data_clean['longitude'], data_clean['latitude'])]
        data_clean = data_clean.drop(["latitude","longitude"],axis = 1)
        self.data_clean = gpd.GeoDataFrame(data_clean, crs="EPSG:4326")
        return self.data_clean




class WF_overview(Wildfire_layer): ## An Overview Layer, which produced a heatmap based on 4 souces
    def __init__(self, source1,source2, source3, source4):
        self.s1 = source1
        self.s2 = source2
        self.s3 = source3
        self.s4 = source4
        self.__heatmap = self.make_heatmap()
        self.layer = self.get_heatmap()


    def make_heatmap(self): ## This Funciton here was created by chat-gpt
        merged_points = pd.concat([self.retrieve_data(self.s1),
                                self.retrieve_data(self.s2),
                                self.retrieve_data(self.s3),
                                self.retrieve_data(self.s4)], ignore_index=True)
        locations = [[point.y, point.x] for point in merged_points.geometry]
        heatmap = Heatmap(locations=locations, radius=20, blur=10, name = "Heatmap", opacity = 0)
        self.timestamp_heatmap = datetime.now()
        return heatmap

    def get_heatmap(self):
        if hasattr(self, "timestamp_heatmap"):
            if (datetime.now() - self.timestamp_heatmap) < timedelta(minutes = 15):
                return self.__heatmap
        return self.make_heatmap()



class Wildfire_map:
    def __init__(self,layer_list):
        m = Map(center=(46.78513, 9.07178), zoom = 6,basemap= basemaps.OpenStreetMap.Mapnik, scroll_wheel_zoom=True)
        m.add(LayersControl())
        legend = {}
        for l in layer_list:
            m.add(l.layer)
            if hasattr(l.layer, "style"):
                legend[l.layer.name] = l.layer.style["fillColor"]
        m.add(LegendControl(legend, position = "topright"))
        self.m = m


In [ ]:
token = "36ece617d2f514d4e90f01418d1b9e28" ##This is my private token

## Create various FIRMS API Objects, with different sensors:
snpp = Firms_source_object(api_token = token, sensor_source= "VIIRS_SNPP_NRT")
modis = Firms_source_object(token, "MODIS_NRT")
noaa20 = Firms_source_object(token, "VIIRS_NOAA20_NRT")
noaa21 = Firms_source_object(token, "VIIRS_NOAA21_NRT")

In [ ]:
layer1 = Wildfire_layer(snpp1, "VIIRIS SNPP Fires", "red")
layer2 = Wildfire_layer(modis, "MODIS Fires", "blue")
#layer3 = Wildfire_layer(noaa20, "NOAA 20 Fires", "green")
#overview = WF_overview(snpp,modis,noaa20,noaa21)

In [ ]:
map1 = Wildfire_map([overview, layer1]).m

In [ ]:
map1

In [ ]:
l1 = layer1.layer
l2 = layer2.layer

In [ ]:
m2 = Map(center=(46.78513, 9.07178), zoom = 6, basemap= basemaps.Esri.WorldTopoMap)
m2.add(l1)
m2.add(l1)
control = LayersControl(position = "topright", collapse = False)
m2.add(control)
m2

In [ ]:
data_snpp["geometry"] = [Point(xy) for xy in zip(data_snpp['longitude'], data_snpp['latitude'])]
gdf = gpd.GeoDataFrame(data_snpp, crs="EPSG:4326")

In [ ]:
api2 = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/IMSR_Incident_Locations_Most_Recent_View/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

response = requests.get(api2)

In [ ]:
gdf = gpd.GeoDataFrame.from_features(response.json()['features'],crs='EPSG:4326')

In [ ]:
snpp1 = Firms_source(api_token = token, sensor_source= "VIIRS_SNPP_NRT")